<a href="https://colab.research.google.com/github/sdam-au/PHI_ETL/blob/master/scripts/XMLS_FROM_GDRIVE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET
import re

import zipfile
import io

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor

In [0]:
# to access sciencedata.dk
!pip install sddk
import sddk
conf = sddk.configure("SDAM_root", "648597@au.dk")

In [0]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Get filenames on Vojtech's gdrive

In [0]:
listed = drive.ListFile({'q': "'1vKecnAiulr9AV-Whl1OiUJFkCH97DieJ' in parents"}).GetList()
#for file in listed:
#  print('title {}, id {}'.format(file['title'], file['id']))

In [27]:
len(listed)

53301

In [0]:
sddk.write_file("SDAM_data/PHI/PHI_gdrive_list.json", listed, conf)

In [36]:
# first entry
listed[0]

GoogleDriveFile({'kind': 'drive#file', 'id': '1yRN3ZKwN30pf0qEnIM28ipfM8WTb42EH', 'etag': '"MTU1Mzc2MjYzNDE0OQ"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1yRN3ZKwN30pf0qEnIM28ipfM8WTb42EH', 'webContentLink': 'https://drive.google.com/uc?id=1yRN3ZKwN30pf0qEnIM28ipfM8WTb42EH&export=download', 'alternateLink': 'https://drive.google.com/file/d/1yRN3ZKwN30pf0qEnIM28ipfM8WTb42EH/view?usp=drivesdk', 'embedLink': 'https://drive.google.com/file/d/1yRN3ZKwN30pf0qEnIM28ipfM8WTb42EH/preview?usp=drivesdk', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/application/xml', 'title': 'PH148104.xml', 'mimeType': 'application/xml', 'labels': {'starred': False, 'hidden': False, 'trashed': True, 'restricted': False, 'viewed': True}, 'copyRequiresWriterPermission': False, 'createdDate': '2019-02-26T14:08:48.547Z', 'modifiedDate': '2019-03-28T08:43:54.149Z', 'modifiedByMeDate': '2019-03-28T08:43:54.149Z', 'lastViewedByMeDate': '2019-03-28T08:43:54.149Z', 'markedViewedByMeDa

In [37]:
# navigate within first entry
listed[0]["webContentLink"]

'https://drive.google.com/uc?id=1yRN3ZKwN30pf0qEnIM28ipfM8WTb42EH&export=download'

In [0]:
listed_ids = [el["id"] for el in listed]

In [53]:
# extract inscription numbers
listed_PHI_numbers = [int(el["title"][2:].partition(".xml")[0]) for el in listed]
listed_PHI_numbers[:10]

[148104, 28034, 80691, 64979, 64978, 80690, 80689, 64977, 64976, 80688]

In [0]:
gdrive_files_dict = dict(zip(listed_PHI_numbers, listed_ids))

In [50]:
listed_PHI_numbers = sorted([int(el) for el in listed_PHI_numbers])
print(listed_PHI_numbers)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [51]:
sddk.write_file("SDAM_data/PHI/vojtechs_inscriptions_list_sorted.json", listed_PHI_numbers, conf)

Your <class 'list'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/vojtechs_inscriptions_list_sorted.json"


In [57]:
# how fast it is to make it useful for reading a large amount of files
%%time
soups = []
for inscription_number in listed_PHI_numbers[:10]:
  content = drive.CreateFile({'id': gdrive_files_dict[inscription_number]}).GetContentString()
  soup = BeautifulSoup(content, "html.parser")
  soups.append(soup)

CPU times: user 294 ms, sys: 11.5 ms, total: 306 ms
Wall time: 15 s


In [42]:
contents[0]

'<!DOCTYPE html>\n<html>\n<head>\n<meta charset="utf-8">\n<meta content="IE=edge" http-equiv="X-UA-Compatible">\n<meta content="" name="description">\n<meta content="width=device-width, initial-scale=1" name="viewport">\n<link href="/images/favicon.ico" rel="shortcut icon">\n<link href="/css/bootstrap.min.css" rel="stylesheet">\n<style>\n        body {\n            padding-top: 20px;\n            padding-bottom: 20px;\n        }\n    </style>\n<link href="/css/bootstrap-theme.css" rel="stylesheet">\n<link href="/css/epig.css?instance-F1516274072540KDCQ5W=_" rel="stylesheet">\n<style>\n    .fontadj { font-size:\n    18\n    px; }\n  </style>\n<title>IG IX,2 240 - PHI Greek Inscriptions</title>\n</head>\n<body data-lift-session-id="F1516279142146Y0TOMO" data-lift-gc="F1516279142147GA3NY5">\n<div id="fsize">\n<span onclick="lift.ajax(&quot;F1516279142148BI21HU=true&quot;, null, null, null)" class="dec" title="Make font smaller"><span aria-hidden="false" class="glyphicon glyphicon-minus-si

# Define date extraction function

In [0]:
### read all inscription numbers within Petra's dataset
inscriptions_list_sorted = sddk.read_file("SDAM_data/PHI/inscriptions_list_sorted.json", "list", conf)
print(inscriptions_list_sorted[:1000])

In [0]:
def date_extractor(datation):
    ### simple cleaning?
    datation = re.sub("\/\d+", "", datation)
    for element in ["?"]:
        datation = datation.replace(element, "")
    for element in ["AD", "Rom. Imp.", "Byzant", "Roman period", "Christian"]:
        if element in datation:
            for element_cent in [("1st", 1, 100),
                            ("2nd", 101, 200),
                            ("3rd", 201, 300),
                            ("4th", 301, 400),
                           ]:
                if element_cent[0] in datation:
                    dating = [datation, element_cent[1], element_cent[2], None, int(np.average([element_cent[1], element_cent[2]])), "cent", "AD"]
                    break
                else:
                    try:
                        post = re.search('(after\s|post\s)(\d+)', datation, flags=re.IGNORECASE).groups()
                        post = int(post[1])
                        dating = [datation, post, None, None, None, "post", "AD"]
                    except:
                        try:
                            date_both = re.search('(\d+)(\-)(\d+)', datation, flags=re.IGNORECASE).groups()
                            post = int(date_both[0])
                            ante = int(date_both[2])
                            dating = [datation, post, ante, None, int(np.average([post, ante])), "range", "AD"]
                        except:
                            try:
                                date_exact = re.search('\d+', datation, flags=re.IGNORECASE)
                                date_exact = int(date_exact.group(0))
                                dating = [datation, None, None, date_exact, date_exact, "exact", "AD"]
                            except:
                                dating = [datation, None, None, None, None, "unprecise", "AD"]
                                
            break
        else:
            for element_cent in [(" V ", -500, -401), 
                            (" V/IV ", -500, -301),
                            (" IV ", -400, -301),
                            (" III ", -300, -201),
                            (" II ", -200, -101),
                            (" I ", -100, -1),
                            ("5th", -500,  -401),
                            ("4th", -400, -301),
                            ("3rd", -300, -201),
                            ("2nd", -200, -101),
                            ("1st", -100, -1)]:
                if element_cent[0] in datation:
                    ### if there as an element indicating century
                    dating = [datation, element_cent[1], element_cent[2], None, int(np.average([element_cent[1], element_cent[2]])), "cent", "BC"]
                    break
                else:
                    try:
                        ante = re.search('(ante\s)(\d+)', datation, flags=re.IGNORECASE).groups()
                        ante = int(ante[1]) * -1
                        dating =  [datation, None, ante, None, None, "ante", "BC"]
                    except:
                        try:
                            post = re.search('(post\s|after\s)(\d+)', datation, flags=re.IGNORECASE).groups()
                            post = int(post[1]) * -1
                            dating = [datation, post, None, None, None, "post", "BC"]
                        except:
                            try:
                                date_both = re.search('(\d+)(\-)(\d+)', datation, flags=re.IGNORECASE).groups()
                                post = int(date_both[0]) * -1 
                                ante = int(date_both[2]) * -1
                                dating = [datation, post, ante, None, int(np.average([post, ante])), "range", "BC"]
                            except:
                                try:
                                    date_exact = re.search('\d+', datation, flags=re.IGNORECASE)
                                    date_exact = int(date_exact.group(0)) * -1
                                    dating = [datation, None, None, date_exact, date_exact, "exact", "BC"]
                                except:
                                        dating = [datation, None, None, None, None, "string", "BC"]
    for reign in [("reign of Hadrian", 117, 138),
                  ("reign of Justinian", 527, 565),
                  ("reign of Ant. Pius", 138, 161),
                  ("reign of Augustus", -27, 19),
                  ("reign of Tiberius", 14, 37)]:
        if reign[0] in datation:
            avr = int(np.average([reign[1], reign[2]]))
            if avr < 0:
                period = "BC"
            else:
                period = "AD"
            dating = [datation, reign[1], reign[2], None, avr, "range", period]
            break
    return dating 

# Define data extraction function

In [0]:
not_date_indicators = ["stoich.", "[", "decree", "Rachi", "Leuko"]
not_date_indicators_2 = ["Krol", "IG"]

def data_extraction(inscription_number):
    content = drive.CreateFile({'id': gdrive_files_dict[inscription_number]}).GetContentString()
    soup = BeautifulSoup(content, "html.parser")
    spans = [span.get_text() for span in soup.find_all("span", class_="name")]
    region_category = spans[1]
    region_precisely = spans[2]
    span_ti = soup.find("span", class_="ti").get_text()
    span_ti = span_ti.split(" — ")
    book = soup.find("a", class_="booklink").get_text()
    datation = span_ti[2]
    for element in not_date_indicators:
            if datation.startswith(element): 
                datation = span_ti[3]
                break
    for element in not_date_indicators_2: # too far
            if datation.startswith(element): 
                datation = span_ti[1]
                break
    raw_text = soup.find("table", class_="grk").get_text()
    dating = list(date_extractor(datation))
    data = [inscription_number, raw_text, book, region_category, region_precisely, span_ti[0], span_ti[1], span_ti] + dating
    return data


In [82]:
data_extraction(1754)

[1754,
 '\n\n\n1\n\n․․5․․οτο— — —\n\n\n\n\n— — —\n\n\n\n\n— — —\n\n\n\n\n․․․ρτ— — —\n\n\n\n5\n\n․․ν ℎο δ․․ε— — —\n\n\n\n\n[․]τον ἐπ[εὶ] εδε— — —\n\n\n\n\n[κ]ατασκ[ευ]έ, ℎο[ι] μὲν τ— — —\n\n\n\n\n[․]ιυτ[․]ς τ[․]ν[․]ε[․]αι προ— — —\n\n\n\n\n[ℎ]οι δὲ βολ̣όμ̣ενοι νευσ— — —\n\n\n\n10\n\n․․ν τέχνει [․]#⁷[․] δὲ σὺν Ἀθ̣[εναίοις(?) — — —]\n\n\n\n\n\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003 — — — ℎιερέος — — —\n\n\n\n\n\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003\u2003 — — —ις να․․․․#⁷— — —\n\n\n',
 'IG I²',
 'Attica (IG I-III)',
 'Attica',
 'Att.',
 'Ath.: Od.Evangelistrias?',
 ['Att.', 'Ath.: Od.Evangelistrias?', 'c. 2nd ac', 'IG I³, p.972'],
 'c. 2nd ac',
 -200,
 -101,
 None,
 -150,
 'cent',
 'BC']

# Test extraction of a limited amount of data

In [83]:
# print gdrive fileid of an inscription from Petra's list on Vojtech gdrive
for inscription_number in inscriptions_list_sorted[:10]:
  print(gdrive_files_dict[inscription_number])

19-gCZ-3xvYLMXP88syDc6MndEKZDilyd
1o1WtJ0D2duTmQjd6QMB5Qrvb_McI6Y9K
19D137av2WYFN0FrD_TLBxQWfp6gZpfyu
1r_pId5bDyhgVuIHgKANyRbeiiB8afviX
1EVIATAtT26qfqp5CaeLCcnIujNtPvCTw
1v6r32XEEN7ye79bXz14cC1gistS233PP
1WjSRRD-RgSQwWgNb0p9hBP-lPR3lMkao
1opzVY2kpGtwWz8qs1ucvZikthNgA5KS8
1pgz0gV7j9Ig4W7vbgg5fGFxXGwEkJo5J
1x8a0EZnwZTtOne5KNLT0yPpOPVCkaucN


In [87]:
%%time
data_list = []
missing_list = []
for inscription_number in inscriptions_list_sorted[:100]: # get inscription numbers from Petra's data
  try:
     data_list.append(data_extraction(inscription_number))
  except:
      try:
        print("new authentification needed")
        drive = GoogleDrive(gauth)
        data_list.append(data_extraction(inscription_number))
      except:
        missing_list.append(inscription_number)
print(missing_list)

new authentification needed
new authentification needed
new authentification needed
new authentification needed
new authentification needed
new authentification needed
new authentification needed
new authentification needed
new authentification needed
new authentification needed
new authentification needed
new authentification needed
new authentification needed
new authentification needed
[1777, 1791, 1792, 1793, 1794, 1795, 1796, 1797, 1798, 1799, 1800, 1802, 1809, 1810]
CPU times: user 3.43 s, sys: 137 ms, total: 3.56 s
Wall time: 2min 21s


In [89]:
inscriptions_df = pd.DataFrame(data_list)
inscriptions_df.columns = ["inscription", "raw_text", "book", "region_cat", "region", "reg_ab", "spec", "notes", "orig_date_text", "post", "ante", "exact", "avr", "type", "period"]
inscriptions_df.head(5)

,inscription,raw_text,book,region_cat,region,reg_ab,spec,notes,orig_date_text,post,ante,exact,avr,type,period
0,1754,\n\n\n1\n\n․․5․․οτο— — —\n\n\n\n\n— — —\n\n\n\...,IG I²,Attica (IG I-III),Attica,Att.,Ath.: Od.Evangelistrias?,"[Att., Ath.: Od.Evangelistrias?, c. 2nd ac, IG...",c. 2nd ac,-200.0,-101.0,NaN,-150.0,cent,BC
1,1755,\n\n\n1\n\n— — —․․5․․ο ℎεκαστ․․․— — —\n\n\n\n\...,IG I²,Attica (IG I-III),Attica,Att.,prov.?,"[Att., prov.?, forgery?, IG I³, p.972]",forgery,NaN,NaN,NaN,NaN,string,BC
2,1756,\n\n\n1\n\nℎοι ℎιππῆς ἀπὸ το͂ν πολεμίον ℎιππαρ...,IG I²,Attica (IG I-III),Attica,Att.,Ath.: Akr.,"[Att., Ath.: Akr., early imp., DAA 135a]",early imp.,NaN,NaN,NaN,NaN,string,BC
3,1757,\n\n\n1\n\n[ℎοι ℎι]π̣π̣ῆ̣[ς ⋮ ἀπὸ] το͂ν ⋮ π[ολ...,IG I²,Attica (IG I-III),Attica,Att.,Ath.: Akr.,"[Att., Ath.: Akr., early imp., DAA 135b]",early imp.,NaN,NaN,NaN,NaN,string,BC
4,1758,\n\n\n1\n\n[–⏕—]δες ἀ[νέθεκε]\n\n\n\n\n[Διὸς κ...,IG I²,Attica (IG I-III),Attica,Att.,Ath.: Akr.,"[Att., Ath.: Akr., [pottery], c. 500-490? BC, ...",c. 500-490 BC,-500.0,-490.0,NaN,-495.0,range,BC


In [90]:
sddk.write_file("SDAM_data/PHI/PHI_Vojtech_sample.json", inscriptions_df, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_Vojtech_sample.json"
